In [1]:
%matplotlib inline

from __future__ import print_function

import io
import numpy as np

from Utilities.loadData import loadTrackFile
from Utilities.config import ConfigParser
from Utilities.track import Track

from PlotInterface.maps import ScatterMapFigure, saveFigure
from IPython.display import Image

import seaborn
seaborn.set_context("poster")



In [8]:
configstr = """
[DataProcess]
InputFile=C:/WorkSpace/data/TC/Allstorms.ibtracs_wmo.v03r06.csv
Source=IBTRACS
StartSeason=1901
FilterSeasons=True

[Region]
; Domain for windfield and hazard calculation
gridLimit={'xMin':90.,'xMax':170.,'yMin':-30.0,'yMax':-5.0}
gridSpace={'x':1.0,'y':1.0}
gridInc={'x':1.0,'y':0.5}

[Input]
landmask = C:/WorkSpace/tcrm/input/landmask.nc
mslpfile = C:/WorkSpace/data/MSLP/slp.day.ltm.nc
datasets = IBTRACS,LTMSLP

[IBTRACS]
; Input data file settings
url = ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r06/wmo/csv/Allstorms.ibtracs_wmo.v03r06.csv.gz
path = C:/WorkSpace/data/TC/
filename = Allstorms.ibtracs_wmo.v03r06.csv
columns = tcserialno,season,num,skip,skip,skip,date,skip,lat,lon,skip,pressure
fielddelimiter = ,
numberofheadinglines = 3
pressureunits = hPa
lengthunits = km
dateformat = %Y-%m-%d %H:%M:%S
speedunits = kph

[LTMSLP]
; MSLP climatology file settings
URL = ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.day.1981-2010.ltm.nc
path = C:/WorkSpace/data/MSLP
filename = slp.day.ltm.nc
"""

In [9]:
config = ConfigParser()
config.readfp(io.BytesIO(configstr))

trackFile = config.get('DataProcess', 'InputFile')
source = config.get('DataProcess', 'Source')
domain = config.geteval('Region', 'gridLimit')

print("Track file: {0}".format(trackFile))
print("Track format: {0}".format(source))
tracks = loadTrackFile(configstr, trackFile, source)
print("There are {0:d} tracks in the input dataset".format(len(tracks)))

In [10]:
xp = []
yp = []

for n, track in enumerate(tracks):
    if track.inRegion(domain):
        xp.append(track.Longitude[0])
        yp.append(track.Latitude[0])
        
data = np.vstack((np.array(xp), np.array(yp)))

In [11]:
xx = np.arange(domain['xMin'], domain['xMax']+0.1,0.1)
yy = np.arange(domain['yMin'], domain['yMax']+0.1,0.1)
[xgrid, ygrid] = np.meshgrid(xx,yy)
mapkwargs = dict(llcrnrlon=domain['xMin'],
                     llcrnrlat=domain['yMin'],
                     urcrnrlon=domain['xMax'],
                     urcrnrlat=domain['yMax'],
                     resolution='h',
                     projection='merc')

startSeason = config.get("DataProcess", "StartSeason")
title = "Genesis points - {0} - 2013".format(startSeason)

In [12]:
fig = ScatterMapFigure()
fig.add(data, xgrid,ygrid, title, [], "", mapkwargs)
fig.plot()

saveFigure(fig, "genesis.png")
Image("genesis.png")